In [6]:
from importlib import reload 
import data_scrub
reload(data_scrub)
from data_scrub import *

pd.set_option('display.max_columns', None)

In [7]:
df = pd.read_csv('data_raw/otodom_privite_2024_03_15.csv')

In [20]:
df = pd.read_csv('data_raw/otodom_last3_2024_03_19.csv')

df.reset_index(drop=True)
df['furnishings_list'] = df['furnishings'].str.replace("'", "").str.split(', ')
exploded_df = df['furnishings_list'].explode()
dummies = pd.get_dummies(exploded_df)
dummies = dummies.groupby(level=0).max()#.astype(bool)
translations = {
    'kuchenka': 'stove',
    'lodówka': 'fridge',
    'meble': 'furniture',
    'piekarnik': 'oven',
    'pralka': 'washing_machine',
    'telewizor': 'television',
    'zmywarka': 'dishwasher'
}
dummies.rename(columns=translations, inplace=True)

df = df.join(dummies)
df[['furnishings', 'stove', 'fridge', 'furniture', 'oven', 'washing_machine', 'television', 'dishwasher']]

,furnishings,stove,fridge,furniture,oven,washing_machine,television,dishwasher
0,"zmywarka, lodówka, piekarnik, kuchenka, pralka",True,True,False,True,True,False,True
1,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",True,True,True,True,True,True,True
2,"lodówka, meble, piekarnik, kuchenka, pralka",True,True,True,True,True,False,False
3,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",True,True,True,True,True,True,True
4,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",True,True,True,True,True,False,True
...,...,...,...,...,...,...,...,...
388,meble,False,False,True,False,False,False,False
389,"zmywarka, lodówka, meble, piekarnik, telewizor...",False,True,True,True,True,True,True
390,meble,False,False,True,False,False,False,False
391,meble,False,False,True,False,False,False,False


In [26]:
df[df.stove.eq(False)].reset_index().link[0]

'https://www.otodom.pl/pl/oferta/garaz-balkon-6m2-ciche-dowynajeciaod10marca24-ID4pI9i'

In [40]:
subways_location = {
    'Plac Wilsona': {'Latitude': 52.26902, 'Longitude': 20.985},
    'Wilanowska': {'Latitude': 52.18098, 'Longitude': 21.02364},
    'Centrum': {'Latitude': 52.23055, 'Longitude': 21.01091},
    'Stokłosy': {'Latitude': 52.15583, 'Longitude': 21.0347},
    'Wierzbno': {'Latitude': 52.18932, 'Longitude': 21.01726},
    'Pole Mokotowskie': {'Latitude': 52.2079, 'Longitude': 21.00837},
    'Swiętokrzyska': {'Latitude': 52.23499, 'Longitude': 21.00835},
    'Ratusz': {'Latitude': 52.24476, 'Longitude': 21.00082},
    'Marymont': {'Latitude': 52.27178, 'Longitude': 20.97232},
    'Imielin': {'Latitude': 52.14952, 'Longitude': 21.04543},
    'Kabaty': {'Latitude': 52.13109, 'Longitude': 21.06604},
    'Ursynów': {'Latitude': 52.16217, 'Longitude': 21.02858},
    'Natolin': {'Latitude': 52.14033, 'Longitude': 21.05776},
    'Bemowo': {'Latitude': 52.2393879, 'Longitude': 20.9175868},
    'Ulrychów': {'Latitude': 52.2402211, 'Longitude': 20.9286984},
    'Księcia Janusza': {'Latitude': 52.238989, 'Longitude': 20.9413589},
    'Młynów': {'Latitude': 52.2370368, 'Longitude': 20.9582514},
    'Płocka': {'Latitude': 52.2331322, 'Longitude': 20.9672229},
    'Rondo Daszyńskiego': {'Latitude': 52.2302503, 'Longitude': 20.9812049},
    'Rondo ONZ': {'Latitude': 52.2326673, 'Longitude': 20.9943419},
    'Nowy Świat Uniwersytet': {'Latitude': 52.2372522, 'Longitude': 21.0163951},
    'Centrum Nauki Kopernik': {'Latitude': 52.2405898, 'Longitude': 21.029495},
    'Stadion Narodowy': {'Latitude': 52.2468026, 'Longitude': 21.0437104},
    'Wileński': {'Latitude': 52.2552165, 'Longitude': 21.0373375},
    'Szwedzka': {'Latitude': 52.2658294, 'Longitude': 21.0477777},
    'Targówek+Mieszkaniowy': {'Latitude': 52.278611, 'Longitude': 21.0478345},
    'Zacisze': {'Latitude': 52.278611, 'Longitude': 21.0478345},
    'Kondratowicza': {'Latitude': 52.2844033, 'Longitude': 21.0489269},
    'Bródno': {'Latitude': 52.2844033, 'Longitude': 21.0489269}}

In [45]:
scrub_scraped_data(df, subways_location)

,rent_price,additional_fees,area,room_num,floor,building_type,extra_space,flat_condition,advertiser_type,students_allowed,furnishings,elevator,parking_space,year_of_construction,additional_information,latitude,longitude,approximate_coordinates,adv_description,link,district,subway_distance,center_distance
0,3570.0,440.0,41.39,2,parter,blok,NaN,do zamieszkania,prywatny,tak,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",tak,garaż/miejsce parkingowe,NaN,NaN,52.181114,20.992173,True,"English below\nBezpośrednio, bez prowizji.\nLo...",https://www.otodom.pl/pl/oferta/2-pokoje-bezpo...,Mokotow,1.938,5.572
1,10000.0,1200.0,55.00,2,3/8,apartamentowiec,balkon,do zamieszkania,prywatny,NaN,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",tak,garaż/miejsce parkingowe,2020.0,"klimatyzacja, piwnica",52.233419,20.980504,False,Nowy apartament 55m2 wykończony i wyposażony w...,https://www.otodom.pl/pl/oferta/premium-aparta...,Wola,0.356,2.198
2,5500.0,800.0,55.00,2,1/8,apartamentowiec,"balkon, taras",do zamieszkania,prywatny,tak,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",tak,garaż/miejsce parkingowe,2019.0,klimatyzacja,52.232445,20.979953,False,Nowoczesny apartament blisko Ronda Daszyńskieg...,https://www.otodom.pl/pl/oferta/2-pokoje-z-gar...,Wola,0.259,2.217
3,3400.0,1000.0,60.00,3,10/10,NaN,balkon,do zamieszkania,prywatny,NaN,"lodówka, meble, piekarnik, kuchenka, pralka",tak,garaż/miejsce parkingowe,NaN,oddzielna kuchnia,52.299660,21.052500,True,"Wynajmę bezpośrednio mieszkanie 3 pokoje, 60m2...",https://www.otodom.pl/pl/oferta/mieszkanie-3-p...,Targowek,1.714,8.248
4,2900.0,NaN,35.00,2,parter/3,kamienica,NaN,do zamieszkania,prywatny,NaN,"lodówka, meble, piekarnik, kuchenka, telewizor...",nie,NaN,1954.0,"piwnica, tylko dla niepalących, oddzielna kuchnia",52.241736,20.989895,False,Wynajmę bezpośrednio/prywatnie (bez prowizji) ...,https://www.otodom.pl/pl/oferta/muranow-nowoli...,Wola,0.816,2.024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,3200.0,800.0,43.00,2,7,blok,balkon,do zamieszkania,prywatny,tak,"lodówka, meble, piekarnik, kuchenka, telewizor...",tak,NaN,NaN,"piwnica, oddzielna kuchnia",52.241681,20.975551,False,Wynajmę mieszkanie 43m2 na bliskiej Woli (ul. ...,https://www.otodom.pl/pl/oferta/mieszkanie-w-c...,Wola,1.107,2.829
2068,4000.0,750.0,51.00,3,7/8,apartamentowiec,balkon,do zamieszkania,prywatny,tak,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",tak,garaż/miejsce parkingowe,2023.0,tylko dla niepalących,52.275791,21.017099,False,"Directly from the owner:\nNew, furnished, very...",https://www.otodom.pl/pl/oferta/3-bedrooms-plu...,Praga_Polnoc,2.114,5.136
2069,2600.0,515.0,28.00,1,1,apartamentowiec,balkon,do zamieszkania,prywatny,NaN,"lodówka, meble, piekarnik, kuchenka, pralka",tak,NaN,2017.0,tylko dla niepalących,52.250201,21.033102,False,EN below\nDzień dobry!\nMam do wynajęcia kawal...,https://www.otodom.pl/pl/oferta/nowe-przestron...,Praga_Polnoc,0.628,2.687
2070,2400.0,NaN,4.00,1,parter/8,apartamentowiec,NaN,do zamieszkania,prywatny,NaN,NaN,tak,garaż/miejsce parkingowe,2024.0,"piwnica, pom. użytkowe",52.227548,20.896628,False,Do wynajęcia bezpośrednio komórka lokatorska n...,https://www.otodom.pl/pl/oferta/komorka-lokato...,Bemowo,1.942,7.875


In [79]:
int = 26

print(df[df.furnishings.isna()].reset_index().link[int])
print(is_furnished_nlp(df[df.furnishings.isna()].reset_index().adv_description[int])) 

https://www.otodom.pl/pl/oferta/bezposrednio-kawalerka-okecie-ochota-super-komuni-ID3m1qv
False


In [75]:
df[df.furnishings.isna()].reset_index().adv_description[int]

'PL/EN BEZPOŚREDNIO/ PRIVATE LISTING\n(English description below)\nWOLNE OD 1 KWIETNIA\nPraga-Południe, ul. Dwernickiego, 2 pokoje, 38 m.kw., oddzielna kuchnia,\nczynsz 2 600 zł\nOferuję do wynajęcia dwupokojowe mieszkanie o powierzchni 38 m.kw. przy ulicy Dwernickiego 19. Kompletnie umeblowane. Pokoje oddzielne, nieprzechodnie, duże. Oddzielna widna kuchnia z oknem. Mieszkanie położone na 1-szym piętrze.\nPowierzchnia całkowita mieszkania to 38 m.kw., na które składają się:\n- pokój nr 1, ok. 12 m.kw.,\n- pokój nr 1, ok. 11 m.kw.,\n- kuchnia, ok. 6 m.kw.\n- łazienka, ok. 4 m.kw.,\n- przedpokój, ok. 5 m.kw.,\n\nBudynek, w którym znajduje się mieszkanie pochodzi z lat pięćdziesiątych. Budynek po wszystkich remontach. Nowa elewacja. Czyste klatki. Przyjazne sąsiedztwo.\nDo mieszkania przynależy piwnica - do użytku najemców.\n\nCzynsz 2 600 zł\nDodatkowe opłaty: woda zimna, ciepła, prąd, gaz, ogrzewanie i śmieci: razem 600 zł przy dwóch osobach miesięcznie.\n\nRAZEM: 3 200 zł\n\nWarsaw, P

In [78]:
import spacy

nlp = spacy.load("pl_core_news_sm")

# Expanded furnishing keywords
furnishing_keywords = [
    'umeblowany', 'umeblowana', 'umeblowane', 'wyposażony', 'wyposażona', 'wyposażone', 'meble', 
    'łóżko', 'sofa', 'szafa', 'pralka'
]

# Basic negation handling
negations = ['nie', 'brak']

def is_furnished_nlp(description):
    doc = nlp(description.lower())
    furnished = False
    for sentence in doc.sents:
        has_furniture_keyword = any(token.lemma_ in furnishing_keywords for token in sentence)
        # has_negation = any(token.lemma_ in negations for token in sentence)
        # If furniture keywords are found without negation, consider it furnished
        if has_furniture_keyword: #and not has_negation:
            furnished = True
    return furnished

In [26]:
print(is_furnished_nlp(df[df.furnishings.isna()].reset_index().adv_description[8]))  # Expected to return True

True
